In [ ]:
import os, csv, json, re
import pandas as pd

import numpy as np
import math

In [ ]:
df = pd.read_pickle("data_cleaned.pkl")
df = df[['input_features', 'pointType_0', 'pointType_1', 'pointType_2',
       'pointType_3', 'pointType_4', 'pointType_5', 'pointType_6',
       'pointType_7', 'pointType_8', 'pointType_9', 'pointType_10',
       'pointType_11', 'pointType_12', 'pointType_13', 'pointType_14',
       'pointType_15', 'pointType_16', 'pointType_17', 'pointType_18',
       'pointType_19', 'pointType_20', 'pointType_21', 'participant_number',
       'left_or_right', 'sample_number']]
print(df)

In [ ]:
print(df)

In [ ]:
def roll_images(input_features_inner, pixels, axis_, point=None):
    # input_features_inner = df.input_features[591] # for debugging
    input_features_inner_np = np.asarray(input_features_inner)
    input_features_inner_np_rolled = np.roll(input_features_inner_np.reshape(64,32),pixels,axis=axis_)
    
    if point is not None:
    # Convert point from string to tuple if needed
        if isinstance(point, str):
            point = tuple(map(float, point[1:-1].split(',')))
        
        if axis_ == 0:
            return [point[0], point[1]+pixels]
        elif axis_ == 1:
            return [point[0]+pixels, point[1]]
        else:
            return [point[0]+pixels, point[1]+pixels]
    else:
        # Flatten the rotated image
        # print("entered here")
        return input_features_inner_np_rolled.reshape(-1)

In [ ]:
x_or_y_shift = 2
axis_to_shift = (0,1)


df['input_features'] = df['input_features'].apply(lambda row: roll_images(row,x_or_y_shift,axis_to_shift))

for i in range(0, 22):
    print(i)
    pointType_col = f'pointType_{i}'
    # Apply the rotation function to each row in the original point column
    df[pointType_col] = df.apply(lambda row: roll_images(row['input_features'], x_or_y_shift,axis_to_shift,point=row[pointType_col]), axis=1)

In [ ]:
df.to_pickle("dataAugmentation_10Degrees.pkl")
df.to_csv("dataAugmentation_10Degrees.csv", index=False)

In [ ]:
def save_sample(df_input_features, df_inner, participant_number, left_or_right, sample_number):
    sampleDir = os.path.normpath("data2/" + participant_number + "/feet/" + left_or_right + "/samples/" + sample_number)
    
    np.savetxt(os.path.join(sampleDir, "augmentedPressure.csv"), df_input_features.reshape(1,2048).round(10),fmt='%f', delimiter=",")
    
    # Create the directory if it doesn't exist
    # os.makedirs(sampleDir, exist_ok=True)
    print(participant_number, left_or_right, sample_number)
    # Convert DataFrame to a list of dictionaries (each dictionary represents a point)
    points_data = []
    for row in df_inner.items():
        # print(row[0],row[1])
        points_data.append({"points": row[1], "pointType": int(re.findall(r'\d+', row[0])[0])})
        # print(points_data)
    
    
    # Write the list of dictionaries to the points.json file
    points_file = os.path.join(sampleDir, "augmentedPoints.json")
    print(points_file)
    with open(points_file, 'w') as file:
        json.dump(points_data, file, indent=2)  # Indent for readability
        

In [ ]:
# Assuming you already have df_final containing your processed data
# You can replace this with your actual DataFrame
for _, row in df.iterrows():
    
    # print(row.input_features.reshape(1,2048))
    # 
    # np.savetxt("foo.csv", row.input_features.reshape(1,2048).round(10),fmt='%f', delimiter=",")
    
    participant_number = row['participant_number']
    left_or_right = row['left_or_right']
    sample_number = row['sample_number']
    selected_columns = [col for col in df.columns if col.startswith("pointType_")]
    df_selected = row[selected_columns]
    
    save_sample(row.input_features, df_selected, participant_number, left_or_right, sample_number)


In [ ]:
# def rotate_image(image_row, angle_degrees, point=None):
#     """
#     Rotate the image by the specified angle (in degrees).
#     Args:
#         image_row (str or np.ndarray): The input image row (flattened 1D array) as a string or NumPy array.
#         angle_degrees (float): The rotation angle in degrees.
#         point (tuple or None): If provided, the original point coordinates (x, y).
#     Returns:
#         np.ndarray or tuple: The rotated image (flattened 1D array) or new coordinates of the rotated point (x_new, y_new).
#     """
#     # Convert image_row from string to NumPy array
#     if isinstance(image_row, list):
#         # image_row = np.fromstring(image_row[1:-1], sep=' ')
#         image_row = np.array(image_row)
# 
#     # Reshape the 1D image row into a 2D image (32x64)
#     image_2d = image_row.reshape(64, 32)
# 
#     # Convert angle to radians
#     angle_radians = math.radians(angle_degrees)
# 
#     # Get image dimensions
#     width, height = image_2d.shape
# 
#     # Calculate the center point
#     center_x, center_y = width//2, height//2
# 
#     # Initialize the rotated image
#     rotated_image = np.zeros_like(image_2d)
# 
#     # Perform rotation
#     for y in range(height):
#         for x in range(width):
#             # Calculate new coordinates after rotation
#             new_x = int((x - center_x) * math.cos(angle_radians) - (y - center_y) * math.sin(angle_radians) + center_x)
#             new_y = int((x - center_x) * math.sin(angle_radians) + (y - center_y) * math.cos(angle_radians) + center_y)
# 
#             # Check if new coordinates are within bounds
#             if 0 <= new_x < width and 0 <= new_y < height:
#                 # print(x,y,new_x,new_y)
#                 rotated_image[new_x, new_y] = image_2d[x, y]
# 
#     if point is not None:
#         # Convert point from string to tuple if needed
#         if isinstance(point, str):
#             point = tuple(map(float, point[1:-1].split(',')))
# 
#         # Calculate new coordinates for the specified point
#         translated_x = point[0] - center_x
#         translated_y = point[1] - center_y
#         new_x_point = translated_x * math.cos(angle_radians) - translated_y * math.sin(angle_radians) + center_x
#         new_y_point = translated_x * math.sin(angle_radians) + translated_y * math.cos(angle_radians) + center_y
#         return [new_x_point, new_y_point]
#     else:
#         # Flatten the rotated image
#         return rotated_image.reshape(-1)

In [ ]:
# def change_angle_to_radius_unit(angle):
#     angle_radius = angle * (np.pi/180)
#     return angle_radius
# 
# def rotate(src_img,angle_of_rotation,pivot_point,shape_img):
# 
#     #1.create rotation matrix with numpy array
#     rotation_mat = np.transpose(np.array([[np.cos(angle_of_rotation),-np.sin(angle_of_rotation)],
#                             [np.sin(angle_of_rotation),np.cos(angle_of_rotation)]]))
#     h,w = shape_img
#     
#     pivot_point_x =  pivot_point[0]
#     pivot_point_y = pivot_point[1]
#     
#     new_img = np.zeros(src_img.shape,dtype='u1') 
# 
#     for height in range(h): #h = number of row
#         for width in range(w): #w = number of col
#             xy_mat = np.array([[width-pivot_point_x],[height-pivot_point_y]])
#             
#             rotate_mat = np.dot(rotation_mat,xy_mat)
# 
#             new_x = pivot_point_x + int(rotate_mat[0])
#             new_y = pivot_point_y + int(rotate_mat[1])
# 
# 
#             if (0<=new_x<=w-1) and (0<=new_y<=h-1): 
#                 new_img[new_y,new_x] = src_img[height,width]
# 
#     return new_img

In [ ]:
# # Example usage:
# # Assuming 'df' is your DataFrame with columns 'image_row' and 'original_point'
# df['input_features'] = df['input_features'].apply(lambda row: rotate_image(row, 5))
# 
# for i in range(0, 22):
#     print(i)
#     pointType_col = f'pointType_{i}'
#     # Apply the rotation function to each row in the original point column
#     df[pointType_col] = df.apply(lambda row: rotate_image(row['input_features'], 5, point=row[pointType_col]), axis=1)
# 
# 
# 
# df['pointType_0'] = df.apply(lambda row: rotate_image(row['input_features'], 5, point=row['pointType_0']), axis=1)

In [ ]:
# def rotate_image(image, angle_degrees, point=None):
#     """
#     Rotate the image by the specified angle (in degrees).
#     Args:
#         image (np.ndarray): The input image (32x64).
#         angle_degrees (float): The rotation angle in degrees.
#         point (tuple or None): If provided, the original point coordinates (x, y).
#     Returns:
#         np.ndarray or tuple: The rotated image (flattened 1D array) or new coordinates of the rotated point (x_new, y_new).
#     """
#     # Convert angle to radians
#     angle_radians = math.radians(angle_degrees)
# 
#     # Get image dimensions
#     height, width = image.shape
# 
#     # Calculate the center point
#     center_x, center_y = width // 2, height // 2
# 
#     # Initialize the rotated image
#     rotated_image = np.zeros_like(image)
# 
#     # Perform rotation
#     for y in range(height):
#         for x in range(width):
#             # Calculate new coordinates after rotation
#             new_x = int((x - center_x) * math.cos(angle_radians) - (y - center_y) * math.sin(angle_radians) + center_x)
#             new_y = int((x - center_x) * math.sin(angle_radians) + (y - center_y) * math.cos(angle_radians) + center_y)
# 
#             # Check if new coordinates are within bounds
#             if 0 <= new_x < width and 0 <= new_y < height:
#                 rotated_image[new_y, new_x] = image[y, x]
# 
#     if point is not None:
#         # Calculate new coordinates for the specified point
#         translated_x = point[0] - center_x
#         translated_y = point[1] - center_y
#         new_x_point = translated_x * math.cos(angle_radians) - translated_y * math.sin(angle_radians) + center_x
#         new_y_point = translated_x * math.sin(angle_radians) + translated_y * math.cos(angle_radians) + center_y
#         return new_x_point, new_y_point
#     else:
#         # Flatten the rotated image
#         return rotated_image.flatten()
# 
# 


In [ ]:
# def load_sample(participant_number, left_or_right, sample_number):
#     sampleDir = os.path.normpath("data2/" + participant_number + "/feet/" + left_or_right + "/samples/" + sample_number)
#     
#     df = pd.DataFrame()
#     
#     # load first row of csv
#     csv_file = os.path.join(sampleDir, "pressure.csv")
#     with open(csv_file, mode='r') as file:
#         csv_reader = csv.reader(file)
#         pressure_data = next(csv_reader)
#         pressure_data = [float(cell) for cell in pressure_data]
# 
#     # Load points
#     points_file = os.path.join(sampleDir, "points.json")
#     with open(points_file, 'r') as file:
#         data = file.read()
#         points_data = json.loads(data)
# 
#     
#     # Populate the DataFrame with columns 'pointType_<number>' and corresponding values
#     df['input_features'] = [pressure_data]
#     for entry in points_data:
#         point_type = entry['pointType']
#         points = entry['points']
#         df[f'pointType_{point_type}'] = [points]
#     
#     return df

In [ ]:
# image_row = np.asarray(df1.input_features.tolist())

In [ ]:
# # Example usage:
# # Assuming 'image_row' is your 1D image row (2048 cells)
# image_2d = image_row.reshape(32, 64)
# 
# # Rotate the entire image clockwise by 10 degrees and flatten it
# clockwise_result = rotate_image(image_2d, 10)
# 
# # Specify a point for rotation (original coordinates)
# original_point = df1.pointType_1.tolist()[0]
# 
# # Calculate new coordinates after clockwise rotation
# clockwise_new_point = rotate_image(image_2d, 10, point=original_point)
# 
# print(f"Clockwise Rotated Image (flattened): {clockwise_result}")
# print(f"New Point Coordinates (Clockwise Rotation): {clockwise_new_point}")